# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>Working with Transformers for Speech Recognition </div></b>


# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>2. Initializing Pipeline</div></b>

In [6]:
from transformers import pipeline

# Initializing a speech recognition pipeline-pretraining model
speech_recognizer = pipeline("automatic-speech-recognition")

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebo

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>3. Installing Dataset</div></b>

In [4]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset
## dataset kütüphanesinde olan bu sesli veri setini kullanıcaz hugging faceden bulduk
# veri setinde 14 dil var inglizceyi istediğim için name kısmına yazdık
#Veri kümesinin yalnızca eğitim bölümünün yükleneceğini belirtir.
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")


README.md:   0%|          | 0.00/5.28k [00:00<?, ?B/s]

minds14.py:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


MInDS-14.zip:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset.features

{'path': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=8000, mono=True, decode=True, id=None),
 'transcription': Value(dtype='string', id=None),
 'english_transcription': Value(dtype='string', id=None),
 'intent_class': ClassLabel(names=['abroad', 'address', 'app_error', 'atm_limit', 'balance', 'business_loan', 'card_issues', 'cash_deposit', 'direct_debit', 'freeze', 'high_value_payment', 'joint_account', 'latest_transactions', 'pay_bill'], id=None),
 'lang_id': ClassLabel(names=['cs-CZ', 'de-DE', 'en-AU', 'en-GB', 'en-US', 'es-ES', 'fr-FR', 'it-IT', 'ko-KR', 'nl-NL', 'pl-PL', 'pt-PT', 'ru-RU', 'zh-CN'], id=None)}

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>4. Data Preprocessing</div></b>

In [9]:
from datasets import Audio

# "Ses" sütununu Ses veri türüne dönüştürme
#Casting the "audio" column to the Audio data type
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))
"""
dataset.cast_column(...): Bu yöntem, bir veri kümesindeki belirli bir sütunun veri tipini değiştirmek için kullanılır.
Burada, "audio" sütunu (muhtemelen ham ses verilerini içerir), modelinizin işleyebileceği özel bir formata dönüştürülmektedir.

Audio(sampling_rate=...): datasets kütüphanesinden gelen Audio tipi burada kullanılmaktadır.
Bu sarmalayıcı, ses verilerine tutarlı bir örnekleme hızı uygular;

speech_recognizer.feature_extractor.sampling_rate: Bu, modelin özellik çıkarıcısının beklediği örnekleme hızını alır
ve ses verilerinin bu gereksinime uygun olarak dönüştürülmesini sağlar.
"""

'\ndataset.cast_column(...): Bu yöntem, bir veri kümesindeki belirli bir sütunun veri tipini değiştirmek için kullanılır.\nBurada, "audio" sütunu (muhtemelen ham ses verilerini içerir), modelinizin işleyebileceği özel bir formata dönüştürülmektedir.\n\nAudio(sampling_rate=...): datasets kütüphanesinden gelen Audio tipi burada kullanılmaktadır.\nBu sarmalayıcı, ses verilerine tutarlı bir örnekleme hızı uygular; \n\nspeech_recognizer.feature_extractor.sampling_rate: Bu, modelin özellik çıkarıcısının beklediği örnekleme hızını alır\nve ses verilerinin bu gereksinime uygun olarak dönüştürülmesini sağlar.\n'

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>5. Speech Recognition</div></b>

In [10]:
result = speech_recognizer(dataset[:4]["audio"]) #(burada 0'dan 3'e kadar)
print([d["text"] for d in result])

['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I FURN A JOINA COUT']


# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>6. Using a Custom Model in the Pipeline</div></b>

In [11]:
from transformers import AutoModelForSequenceClassification

# Initializing a model for sequence classification
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name) #modelimize göre bir tokenizer tanımlamasını istedim
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer) #classifieri uygulamamızı sağlayan araç gibi düşün

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>7. Prediction</div></b>

In [14]:
classifier("C'est un film génial. J'aime ça.")

[{'label': '5 stars', 'score': 0.7349995970726013}]

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>8. How to Work AutoTokenizer</div></b>

In [15]:
encoding = tokenizer("This film is nice. I liked it.")
print(encoding)
#tokenizer, cümleyi daha küçük (token'lara) ayırır ve her bir token'ı sayısal bir kimlik (token ID) ile eşleştirir.
#encoding değişkeninin çıktısı genellikle sözlük (dictionary) olacaktır:

{'input_ids': [101, 10372, 10388, 10127, 24242, 119, 151, 11531, 10163, 10197, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [19]:
batch = tokenizer(
    ["I like NLP. This is an awosome area", "We hope you don't hate it."],
    max_length=512,
    truncation=True,
    padding=True,
    return_tensors="pt")
"""
bir tokenizer kullanarak iki cümleyi toplu olarak (batch) encode ediyorsunuz.
Bu işlem, her cümle için token'ları sayısal kimliklere dönüştürmeyi ve onları uygun bir formata sokmayı içerir.
max_length=512: Token sayısı 512'yi aşarsa kesilecek (truncation).
truncation=True: Eğer bir cümle 512 token'dan uzunsa, fazlasını keser.
padding=True: Cümleler arasındaki uzunluk farkını gidermek için daha kısa cümleleri doldurur.
return_tensors="pt": Çıktının PyTorch tensörü (tensor) formatında olmasını sağlar.
return_tensors="pt" parametresini kullanmazsanız,
tokenizer çıktısı standart Python veri yapıları (genellikle listeler ve numpy dizileri) olarak dönecektir.

Dizilerin ve Tensörlerin Farkı
Diziler: Listeler, Python dilinde dinamik olarak büyüyebilen ve farklı veri türlerini içerebilen temel veri yapılarıdır.
Ancak, işlem yaparken tüm öğelerin aynı veri türünde olmasını beklemek zorunda kalabilirsiniz.
Tensörler: Tensor'lar(PyTorch veya TensorFlow kullanıyorsanız), çok boyutlu dizilerdir ve genellikle sayısal hesaplamalar için optimize edilmiştir.
Tensörler, GPU üzerinde daha hızlı işlem yapma yeteneği sunar ve derin öğrenme uygulamalarında yaygın olarak kullanılır.
Eğer modelinizi PyTorch ile eğitiyorsanız veya kullanıyorsanız,
tensör formatında döndürmek, model girdi verilerinizi doğrudan kullanmanıza olanak tanır.
Bu, dönüşüm aşamasında zaman kaybını azaltır.
Eğer return_tensors parametresini kullanmazsanız, verileri tensör formatına dönüştürmek için ek bir adım atmanız gerekecektir.
"""

'\nbir tokenizer kullanarak iki cümleyi toplu olarak (batch) encode ediyorsunuz.\nBu işlem, her cümle için token\'ları sayısal kimliklere dönüştürmeyi ve onları uygun bir formata sokmayı içerir.\nmax_length=512: Token sayısı 512\'yi aşarsa kesilecek (truncation).\ntruncation=True: Eğer bir cümle 512 token\'dan uzunsa, fazlasını keser.\npadding=True: Cümleler arasındaki uzunluk farkını gidermek için daha kısa cümleleri doldurur.\nreturn_tensors="pt": Çıktının PyTorch tensörü (tensor) formatında olmasını sağlar.\nreturn_tensors="pt" parametresini kullanmazsanız,\ntokenizer çıktısı standart Python veri yapıları (genellikle listeler ve numpy dizileri) olarak dönecektir. \n\nDizilerin ve Tensörlerin Farkı\nDiziler: Listeler, Python dilinde dinamik olarak büyüyebilen ve farklı veri türlerini içerebilen temel veri yapılarıdır.\nAncak, işlem yaparken tüm öğelerin aynı veri türünde olmasını beklemek zorunda kalabilirsiniz.\nTensörler: Tensor\'lar(PyTorch veya TensorFlow kullanıyorsanız), çok bo

In [20]:
batch

{'input_ids': tensor([[  101,   151, 11531, 19848, 10373,   119, 10372, 10127, 10144, 37079,
         20104, 10688, 10793,   102],
        [  101, 11312, 18763, 10855, 11530,   112,   162, 39487, 10197,   119,
           102,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>8. How to Work AutoModel</div></b>

In [24]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [22]:
outputs = model(**batch)
"""
**batch kullanımı, batch içindeki anahtar-değer çiftlerini modelin argümanları olarak geçirir.
Yani, batch sözlüğündeki her bir anahtar, modelin parametrelerine doğrudan eşlenir.
Bu kullanım, modelin beklediği argümanların (örneğin input_ids, attention_mask, token_type_ids) doğrudan verilmesini sağlar.
"""

In [23]:
from torch import nn
# Applying softmax to model logits to get predictions
predictions = nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
""""
bir modelin çıkış logits'lerine softmax uygulayarak olasılık tahminleri elde etmek için kullanılır.
Softmax, çok sınıflı sınıflandırma problemlerinde kullanılan bir matematiksel fonksiyondur.
Bir dizi gerçek sayıyı (logits) alır ve bu sayıları, toplamı 1 olan bir olasılık dağılımına dönüştürür.
Örneğin, modelin birden fazla sınıf arasında seçim yapması gerektiğinde, softmax bu sınıfların her biri için tahmin edilen olasılıkları hesaplar.
logits, modelin son katmanından çıkan, sınıflandırma için ham değerlerdir.
Bu değerler genellikle negatif veya pozitif sonsuzluğa yakın değerler olabilir ve doğrudan olasılık olarak yorumlanamazlar.
nn.functional.softmax(...): Softmax fonksiyonu, logits'leri olasılıklara dönüştürmek için kullanılır.
Softmax, verilen bir girdi dizisindeki her bir değerin, tüm değerler toplamına oranını alarak, her bir değeri 0 ile 1 arasında normalize eder.
dim=-1: Bu parametre, softmax işleminin hangi boyutta uygulanacağını belirtir.
dim=-1 kullanıldığında, son boyutta softmax uygulanır. Bu, genellikle sınıf sayısını temsil eder.

Örnek Çıktı
tensor([[2.0, 1.0, 0.1], [0.5, 1.5, 2.0]])
Softmax uygulandığında, predictions şu şekilde olabilir:
tensor([[0.6590, 0.2424, 0.0986],
        [0.0900, 0.6652, 0.2448]])
Bu durumda, her satır, sınıf olasılıklarını temsil eder ve her satırdaki değerler toplamı 1'e eşittir.
Bu işlem, sınıflandırma görevlerinde tahminlerin yorumlanmasını kolaylaştırır, çünkü çıktılar artık belirli bir sınıfa ait olma olasılıklarını temsil eder.

"""

tensor([[0.2105, 0.1720, 0.2181, 0.1997, 0.1997],
        [0.2084, 0.1826, 0.1969, 0.1755, 0.2365]], grad_fn=<SoftmaxBackward0>)


# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>9. Saving the Model</div></b>

In [25]:
save_directory = "./save_pretrained"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [26]:
#kaydettiğin modeli kullanmak istersende su sekilde kullanırsın
model= AutoModelForSequenceClassification.from_pretrained("./save_pretrained")